In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import cos,sin,deg2rad
import math
import skimage.draw
import skimage.color
import os
from skimage import io
import pandas as pd
from skimage.transform import rescale

In [ ]:
# Funções para operações geométricas passadas na aula

def translacao (tx, ty):

    return np.array ([[1,0,tx], \
                      [0,1,ty], \
                      [0,0,1]]).astype('float')

def escalonamento (sx, sy):
    return np.array ([[sx,0,0], \
                      [0,sy,0], \
                      [0,0,1]]).astype('float')

def rotacao (theta):
    theta = deg2rad (theta)
    c = cos (theta)
    s = sin (theta)
    return np.array ([[c,-s,0], \
                      [s,c,0], \
                      [0,0,1]]).astype('float')

In [ ]:
# Leitura dos códigos de barras, converção de escala para tons de cinza e converção inteiro
img = (io.imread ('codigo_de_barras_10.png',as_gray=True)*255).astype('uint8') # Para as imagens normais, se rotação
#img = (io.imread ('codigo_rotacionado_0.jpg').astype('uint8')) # Para as imagens rotacionadas

# Exibe uma única imagem
plt.figure(figsize=(5, 5))  # Ajustar o tamanho da figura
plt.imshow(img, cmap='gray',vmin=0,vmax=255)  # Mostrar a imagem em escala de cinza
plt.title('Imagem de entrada')  # Definir o título
plt.show()
print(f"Imagem.shape[0] = {img.shape[0]}, Imagem.shape[1] = {img.shape[1]}") # Apresentando o tamanho da imagem

In [ ]:
# Pegando a linha central para detectar as bordas horizontais
linha_central = img[img.shape[0] // 2, :]  # Pegamos a linha do meio

# Encontrar a primeira e última coluna com pixel preto
col_ini = np.argmax(linha_central == 0) # Primeira ocorrência de preto
col_fim = (len(linha_central) - np.argmax(linha_central[::-1] == 0)) # Última ocorrência de preto

# Cortar a imagem apenas na região útil
img = img[:, col_ini:col_fim]  # Mantemos todas as linhas, mas cortamos colunas

# Mostrar a imagem cortada
plt.imshow(img, cmap='gray')
plt.title('Código de Barras Sem Bordas Brancas')
plt.show()

In [ ]:
# Exibe o histograma da imagem
plt.figure(figsize=(5, 5))  # Ajustar o tamanho da figura
h = np.histogram (img,bins=256) # Calcula o histograma
plt.plot (h[0],'-k')
plt.title('Histograma')  # Definir o título
plt.show()

Tomando a atividade 2 como exemplo, na qual é usada a biblioteca do skimage para mudar a escala das placas de carros:

<p style="text-align:center;">img_bin_comprimida= rescale(img, scale_x, anti_aliasing=True)</p>

foi pensada uma estratéria para pegar o códido de barras, de forma simplificada, já que a imagem não tem dimensõs de 1:1 com relação as barras, o que causa problemas na leitura caso não seja corrigido. Podem haver outros métodos, porém a equipe julgou esse como o mais rápido e efeitvo de ser implementado com base nos materiais disponibilizados anteriormente.

In [ ]:
# Obtem a largura da imagem
largura_original = img.shape[1]

# Estima o número de barras analisando as transições preto-branco
linha_meio = img[img.shape[0] // 2, :]  # Pega a linha central

# Verifica as transições

# 1) linha_meio < 1 => Isso cria um array booleano onde True representa pixels pretos (valores menores que 1) e False representa
#    pixels brancos (valores maiores que 1).
# 2) np.diff(linha_meio < 1) => A função np.diff() calcula a diferença entre elementos consecutivos no array booleano.
# 3) np.count_nonzero(np.diff(linha_meio < 1)) => conta quantos valores diferentes de zero existem. Como cada transição
#    (preto → branco ou branco → preto) gera um 1 ou -1, essa contagem nos dá o número total de transições no código de barras.
transicoes = np.count_nonzero(np.diff(linha_meio < 1))  # Contar mudanças P/B

# Define o fator de escala horizontal para que cada barra tenha 1 pixel de largura
scale_x = transicoes / largura_original  # Fator de escala na largura

# Aplica o rescale mantendo a altura (scale_y = 1)
img_bin_comprimida = rescale(img, (1, scale_x), anti_aliasing=False, order=0, preserve_range=True)

# Converte para uint8, e binarizando para exibição correta
img_bin_comprimida = (img_bin_comprimida * 255).astype(np.uint8)

In [ ]:
print(img_bin_comprimida)

In [ ]:
# Mostrar a imagem comprimida
plt.imshow(img_bin_comprimida, cmap='gray')
plt.title('Código de Barras com 1 pixel por Barra')
plt.show()

In [ ]:
print(f"Imagem.shape[0] = {img_bin_comprimida.shape[0]}, Imagem.shape[1] = {img_bin_comprimida.shape[1]}") # Apresentando o tamanho da imagem

In [ ]:
# Encontrar o índice da linha central
linha_central = img_bin_comprimida.shape[0] // 2  # Divide por 2 para pegar o meio

In [ ]:
# Pegar apenas a linha central e todas as suas colunas
codigo = img_bin_comprimida[linha_central, :]

Se código foi digitado manualmente, como um conjunto de zeros e uns, em uma array numpy, no formato code 128, não importando o tamanho, desde que tenha 11 elementos por palavras, ele será reconhecido. Essa é a primeira parte do entregável.

In [ ]:
print(f"Código encontrado: {codigo}")
print(f"Tamanho do código encontrado: {len(codigo)}")

In [ ]:
# Exibe a linha como uma imagem
plt.plot(codigo, '-k')  # Plota a intensidade da linha
plt.show()

In [ ]:
codigo = 1 - codigo # Calcula o negativo, pois código de barras é em nagetivo, todos os valores comaçam com uma barra de zeros
                    # que são lidos como uns

In [ ]:
print(f"Código encontrado: {codigo}")
print(f"Tamanho do código encontrado: {len(codigo)}")

In [ ]:
# Exibe a linha como uma imagem
plt.plot(codigo, '-k')  # Plota a intensidade da linha
plt.show()

In [ ]:
# Concatena o array, tornando-o uma string, o que retira as vírgulas
str_codigo = "".join((codigo.astype(int)).astype(str))

In [ ]:
print(str_codigo) # Printa o código

A tabela de referências foi gerada usando o códigos de barras code 128, e consta em anexo junto a atividade, nomeado "AtvFINAL-Gabriel_Lima_Dantas-Rafael_Lima_Dantas-t03 - Tabela code128 completa.xlsx"

A tabela é só de um caractere, assim temos cerca de 119 opções

In [ ]:
tabela = pd.read_excel('AtvFINAL-Gabriel_Lima_Dantas-Rafael_Lima_Dantas-t03 - Tabela code128 completa.xlsx', engine='openpyxl')

In [ ]:
# Separar as colunas em variáveis separadas
referencia = tabela['Padrão Code 128'].to_numpy()  # Substitua 'nome_da_coluna_numerica' pelo nome correto da coluna
mensagem = tabela['Caractere'].to_numpy()    # Substitua 'nome_da_coluna_strings' pelo nome correto da coluna

In [ ]:
palavras = [] # Cria uma lista para as palavras, os conjuntos de 11 dígitos que formam o código de barras
grupo = 0 # Contado para separar as palavras
for e in range(len(str_codigo)//11): # O //11, evita que a lista tenha elementos vazios
    palavras.append(str_codigo[grupo:grupo+11]) # Juntas os conjuntos na lista
    grupo = grupo + 11 # Pula a cada 11 dígitos

In [ ]:
#palavras = [item for item in palavras if item != ""] # Retira os elementos vazios da lista

In [ ]:
print(palavras) # Printa a lista com as palavras

Ao encontrar o código correspondente, é impresso o caractere associado. O enumerate() acessa tanto o valor de referencia quanto o índice, e permite imprimir o caractere correspondente a mensagem[i].

In [ ]:
for p in palavras:  # Compara a lista com a tabela
    for i, confere in enumerate(referencia): 
        if confere == int(p): # Passa novamente para int, a fim de comparar com a tabela
            print(f"Encontrado: {mensagem[i]}") # Apresenta o resultado das comparações realizadas            